In [1]:
pip show tweepy


Note: you may need to restart the kernel to use updated packages.


The filename, directory name, or volume label syntax is incorrect.


In [2]:
#conda install -c conda-forge "tweepy>=4"


In [3]:
#conda install -c conda-forge bs4


In [4]:
# for the twitter section
import tweepy
import os
import datetime
import re
from pprint import pprint

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter

In [5]:
import tweepy


In [6]:
api_key = 'QiCYykYDGdcRWz42kWOc9mF2B'
api_secret_key = '1pDkB2gCuePvPn9pflkTVsPIoZ2OcjNM6ZWNTphiIW1zllCLUf'
auth = tweepy.AppAuthHandler(api_key, api_secret_key)
api = tweepy.API(auth)
print ('API Host', api.host)
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAA4lhAEAAAAAw5dLvRH3wCG4kgN7AmpUKoFXhYI%3D3BcVDYA8Uz5r3bZuHlF5I1ULM0J8xjeTwjdj7jJ9GPXDMU7OIh'


API Host api.twitter.com


In [7]:
#from api_keys import api_key, api_key_secret, bearer_token


In [8]:
client = tweepy.Client(bearer_token,wait_on_rate_limit=True)


In [9]:
handle = "37chandler"
user_obj = client.get_user(username=handle)

followers = client.get_users_followers(
    # Learn about user fields here: 
    # https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/user
    user_obj.data.id, user_fields=["created_at","description","location",
                                   "public_metrics"]
)

In [11]:
num_to_print = 20

for idx, user in enumerate(followers.data) :
    following_count = user.public_metrics['following_count']
    followers_count = user.public_metrics['followers_count']
    
    print(f"{user.name} lists '{user.location}' as their location.")
    print(f" Following: {following_count}, Followers: {followers_count}.")
    print()
    
    if idx >= (num_to_print - 1) :
        break
    

Dave Renn lists 'None' as their location.
 Following: 45, Followers: 10.

Lionel lists 'None' as their location.
 Following: 202, Followers: 204.

Megan Randall lists 'None' as their location.
 Following: 141, Followers: 100.

Jacob Salzman lists 'None' as their location.
 Following: 562, Followers: 134.

twiter not fun lists 'None' as their location.
 Following: 221, Followers: 21.

Hariettwilsonincarnate lists 'None' as their location.
 Following: 219, Followers: 61.

Christian Tinsley lists 'None' as their location.
 Following: 3, Followers: 0.

Steve lists 'I'm over here.' as their location.
 Following: 1592, Followers: 33.

John O'Connor 🇺🇦 lists 'None' as their location.
 Following: 8, Followers: 1.

CodeGrade lists 'Amsterdam' as their location.
 Following: 2817, Followers: 426.

Cleverhood lists 'Providence, RI' as their location.
 Following: 2795, Followers: 3563.

Regina 🚶‍♀️🚲🌳 lists 'Minneapolis' as their location.
 Following: 2802, Followers: 3342.

Eric Hallstrom lists 'Mi

In [10]:
max_followers = 0

for idx, user in enumerate(followers.data) :
    followers_count = user.public_metrics['followers_count']
    
    if followers_count > max_followers :
        max_followers = followers_count
        max_follower_user = user

        
print(max_follower_user)
print(max_follower_user.public_metrics)

WedgeLIVE
{'followers_count': 14220, 'following_count': 2223, 'tweet_count': 56250, 'listed_count': 218}


In [11]:
response = client.get_user(id=user_obj.data.id,
                          user_fields=["created_at","description","location",
                                       "entities","name","pinned_tweet_id","profile_image_url",
                                       "verified","public_metrics"])

In [14]:
for field, value in response.data.items() :
    print(f"for {field} we have {value}")

for description we have He/Him. Data scientist, urban cyclist, educator, erstwhile frisbee player. 

¯\_(ツ)_/¯
for created_at we have 2009-04-18 22:08:22+00:00
for public_metrics we have {'followers_count': 192, 'following_count': 589, 'tweet_count': 997, 'listed_count': 3}
for username we have 37chandler
for id we have 33029025
for profile_image_url we have https://pbs.twimg.com/profile_images/2680483898/b30ae76f909352dbae5e371fb1c27454_normal.png
for name we have John Chandler
for verified we have False
for location we have MN


Q: How many fields are being returned in the response object?

A: There are 10 fields being returned in the response object

Q: Are any of the fields within the user object non-scalar? (I.e., more complicated than a simple data type like integer, float, string, boolean, etc.)
 
A: public_metrics, profile_image_url, and created_at are non-scalar

Q: How many friends, followers, and tweets does this user have?

A: 193 followers, 996 tweets, 589 friends

In [15]:
response = client.get_users_tweets(user_obj.data.id)

# By default, only the ID and text fields of each Tweet will be returned
for idx, tweet in enumerate(response.data) :
    print(tweet.id)
    print(tweet.text)
    print()
    
    if idx > 10 :
        break

1569760631548690437
RT @dtmooreeditor: So there's a particular quirk of English grammar that I've always found quite endearing: the exocentric verb-noun compou…

1569155273742327811
As a Minneapolis person, I knew we had Toronto beat, but I didn't realize Portland had us beat: https://t.co/xrx5mOFcWK.

But @nytimes, c'mon! https://t.co/M9mBWhdgsj

1568982292923826176
RT @wonderofscience: Amazing lenticular cloud over Mount Fuji

Credit: Iurie Belegurschi
https://t.co/0mUxl28H9U

1568242374085869570
RT @depthsofwiki: lots of memes about speedy wikipedia editors — quick thread about what went down on wikipedia in the minutes after her de…

1568074978754703361
@DrLaurenWilson @leighradwood @MaritsaGeorgiou @Walgreens I could not possibly agree more with this sentiment. Compared to almost any other primary care I've received, they are great.

1567530169686196224
@DrLaurenWilson @MaritsaGeorgiou @Walgreens For those who have access to Curry Health Center on campus, you can get a bivalent bo

In [16]:
handle_followers = []
pulls = 0
max_pulls = 100
next_token = None

while True :

    followers = client.get_users_followers(
        user_obj.data.id, 
        max_results=25*40, # when you do this for real, set this to 1000!
        pagination_token = next_token,
        user_fields=["created_at","description","location",
                     "entities","name","pinned_tweet_id","profile_image_url",
                     "verified","public_metrics"]
    )
    pulls += 1
    
    for follower in followers.data : 
        follower_row = (follower.id,follower.name,follower.created_at,follower.description)
        handle_followers.append(follower_row)
    
    if 'next_token' in followers.meta and pulls < max_pulls :
        next_token = followers.meta['next_token']
    else : 
        break
    

In [17]:
handle_followers

[(1553891535040630785,
  'Dave Renn',
  datetime.datetime(2022, 7, 31, 23, 53, 28, tzinfo=datetime.timezone.utc),
  ''),
 (618734476,
  'Lionel',
  datetime.datetime(2012, 6, 26, 4, 35, 47, tzinfo=datetime.timezone.utc),
  ''),
 (401835058,
  'Megan Randall',
  datetime.datetime(2011, 10, 31, 4, 54, 16, tzinfo=datetime.timezone.utc),
  ''),
 (634650371,
  'Jacob Salzman',
  datetime.datetime(2012, 7, 13, 15, 27, 27, tzinfo=datetime.timezone.utc),
  'I play a little ultimate. Eat, sleep, wheat. I get 🍞\n\nhe/him'),
 (1227344433622003712,
  'twiter not fun',
  datetime.datetime(2020, 2, 11, 21, 31, 41, tzinfo=datetime.timezone.utc),
  'Musician, writer and producer based in limerick'),
 (2989225576,
  'Hariettwilsonincarnate',
  datetime.datetime(2015, 1, 21, 7, 35, 1, tzinfo=datetime.timezone.utc),
  'If you are depressed you are living in the past. If you are anxious you are living in the future. If you are at peace you are living in the present. Lao Tzu'),
 (1478401670274957312,
  'Ch

In [12]:
artists = dict()

for handle in ['youngthug','gucci1017'] : 
    user_obj = client.get_user(username=handle,user_fields=["public_metrics"])
    artists[handle] = (user_obj.data.id, 
                       handle,
                       user_obj.data.public_metrics['followers_count'])
    

for artist, data in artists.items() : 
    print(f"It would take {data[2]/(1000*15*4):.2f} hours to pull all {data[2]} followers for {artist}. ")

It would take 101.39 hours to pull all 6083518 followers for youngthug. 
It would take 156.97 hours to pull all 9418026 followers for gucci1017. 


In [13]:
# Make the "twitter" folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then "unlink" it. Then create a new one.

if not os.path.isdir("twitter") : 
    #shutil.rmtree("twitter/")
    os.mkdir("twitter")

In [14]:
num_followers_to_pull = 100*1000 # feel free to use this to limit the number of followers you pull.


In [15]:
path = 'twitter'


In [26]:
  def conversion(follower):
            user_id = str(follower.id)
            username = str(follower.username)
            name = str(follower.name)
            location = str(follower.location)
            followers_count = str(follower.public_metrics['followers_count'])
            following_count = str(follower.public_metrics['following_count'])
    
            description = follower.description
            description = description.replace("\n" , "|")
            description = description.replace("\t" , "  ")
            description = description.replace("!" , "``")

In [ ]:
# Modify the below code stub to pull the follower IDs and write them to a file. 

handles = ['youngthug','gucci1017']
whitespace_pattern = re.compile(r"\s+")

user_data = dict() 
followers_data = dict()


 
for handle in handles :
    user_data[handle] = [] # will be a list of lists
    followers_data[handle] = [] # will be a simple list of IDs
      # Create the output file names 
    
    followers_output_file = handle + "_followers.txt"
    user_data_output_file = handle + "_follower_data.txt"
    
    user_obj = client.get_user(username = handle)
    print(user_obj)

    followers_output_file_path = os.path.join(path,followers_output_file)
    user_data_output_file_path = os.path.join(path,user_data_output_file)

# Using tweepy.Paginator (https://docs.tweepy.org/en/latest/v2_pagination.html), 
# use `get_users_followers` to pull the follower data requested. 

    
    for followers in tweepy.Paginator(client.get_users_followers, user_obj.data.id, user_fields = ["description","location","public_metrics"],
                                max_results=100, limit=10):
                 
        for follower in followers.data:
            
            user_data_rows = [follower.username, follower.name, follower.id, follower.location, 
                      follower.public_metrics['followers_count'], follower.public_metrics['following_count']
                     ,re.sub(r'\s+', '', follower.description).strip()]
            user_data[handle].append(user_data_rows)
            followers_data[handle].append(follower.id)
        
        def conversion(follower):
            user_id = str(follower.id)
            username = str(follower.username)
            name = str(follower.name)
            location = str(follower.location)
            followers_count = str(follower.public_metrics['followers_count'])
            following_count = str(follower.public_metrics['following_count'])
    
            description = follower.description
            description = description.replace("\n" , "|")
            description = description.replace("\t" , "  ")
            description = description.replace("!" , "``")
        
        print(followers_data[handle])
        print(user_data[handle])
        
        
        with open(followers_output_file,'w') as f1:
              for follower in followers_data[handle]:
                    f1.write(str(follower))
        with open(followers_output_file,'w') as f2:
              for follower in followers_data[handle]:
                    f2.write(str(follower))
                 
       
       

Response(data=<User id=238763290 name=Young Thug  ひ username=youngthug>, includes={}, errors=[], meta={})
[1534963964236468224, 1570787133517864963, 1570768652424585218, 1570788561833566215, 1559989332726915073, 1368884283465687043, 3416743031, 1545279415327883266, 1570134873532243973, 1486937239301111812, 1529233399663063040, 1528036410095046656, 1480157225314394117, 1513763475968778240, 1495488856824635397, 1570778851441086469, 1469332049798569989, 1563246429711519750, 1508207877114286084, 1561788119120322561, 1570777338534678528, 1570393609265872896, 1564902708045283332, 1263252149900894208, 1570784941448765442, 408908404, 1569505203891892224, 1570786187194892289, 1545117268807483395, 1567549221523394561, 1210437027801501697, 1242332033352757249, 1517323200149000192, 1340765324162052098, 1414604495519109124, 1549591116315447296, 1570759473278459906, 1569083018467221507, 1537876304132988929, 1570779972226121731, 1570644585960833024, 1570783152972632065, 1484132909548199936, 154704288

Rate limit exceeded. Sleeping for 850 seconds.


In [24]:
print(dir(user_data))

['__class__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'clear', 'copy', 'fromkeys', 'get', 'items', 'keys', 'pop', 'popitem', 'setdefault', 'update', 'values']


In [37]:
followers_output_file

'gucci1017_followers.txt'

In [40]:
user_data_output_file

'gucci1017_follower_data.txt'

In [39]:
# For each response object, extract the needed fields and store them in a dictionary or data frame. 
followers_data[handle] 
user_data[handle]
print({len(followers)})
       
    
    

{4}


In [33]:
f1

<_io.TextIOWrapper name='youngthug_followers.txt' mode='w' encoding='cp1252'>

In [41]:
f2

<_io.TextIOWrapper name='gucci1017_followers.txt' mode='w' encoding='cp1252'>

In [28]:
tricky_description = """
    Home by Warsan Shire
    
    no one leaves home unless
    home is the mouth of a shark.
    you only run for the border
    when you see the whole city
    running as well.

"""
# This won't work in a tab-delimited text file.

clean_description = re.sub(r"\s+"," ",tricky_description).strip()
clean_description

'Home by Warsan Shire no one leaves home unless home is the mouth of a shark. you only run for the border when you see the whole city running as well.'

# Part 1

In [42]:
artists = {'guccimane':"https://www.azlyrics.com/g/guccimane.html",
           'youngthug': "https://www.azlyrics.com/y/youngthug.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

Q: Take a look at the robots.txt page on www.azlyrics.com. (You can read more about these pages here.) Is the scraping we are about to do allowed or disallowed by this page? How do you know?

A: https://www.azlyrics.com/robot.txt defines which urls that the search engine urls can access from; it is allowed by this page

In [43]:
import random
import requests
from bs4 import BeautifulSoup

In [44]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())
    
    # now extract the links to lyrics pages from this page
    soup = BeautifulSoup(r.content)
    urls = soup.select('a[href^="/lyrics/"]')
    
    # store the links `lyrics_pages` where the key is the artist and the value is a list of links.
    lyrics_pages[artist] = ["https://www.azlyrics.com" + url.get("href") for url in urls] 


In [36]:
print(urls)

[<a href="/lyrics/youngthug/intro.html" target="_blank">Intro</a>, <a href="/lyrics/youngthug/jungle648228.html" target="_blank">Jungle</a>, <a href="/lyrics/youngthug/weare.html" target="_blank">We Are</a>, <a href="/lyrics/youngthug/eatualive.html" target="_blank">Eat U Alive</a>, <a href="/lyrics/youngthug/ballonyall.html" target="_blank">Ball On Yall</a>, <a href="/lyrics/youngthug/dropbars.html" target="_blank">Drop Bars</a>, <a href="/lyrics/youngthug/spartan.html" target="_blank">Spartan</a>, <a href="/lyrics/youngthug/zanman.html" target="_blank">Zan Man</a>, <a href="/lyrics/youngthug/indisbitch.html" target="_blank">In Dis Bitch</a>, <a href="/lyrics/youngthug/rip.html" target="_blank">Rip</a>, <a href="/lyrics/youngthug/achieve.html" target="_blank">Achieve</a>, <a href="/lyrics/youngthug/weballin.html" target="_blank">We Ballin'</a>, <a href="/lyrics/youngthug/summer.html" target="_blank">Summer</a>, <a href="/lyrics/youngthug/haitislang.html" target="_blank">Haiti Slang</a

In [37]:
lyrics_pages.items()

dict_items([('guccimane', ['https://www.azlyrics.com/lyrics/guccimane/intro1029386.html', 'https://www.azlyrics.com/lyrics/guccimane/fosho.html', 'https://www.azlyrics.com/lyrics/guccimane/payme.html', 'https://www.azlyrics.com/lyrics/guccimane/12.html', 'https://www.azlyrics.com/lyrics/guccimane/saycheese.html', 'https://www.azlyrics.com/lyrics/guccimane/musclesinmyhand.html', 'https://www.azlyrics.com/lyrics/guccimane/guccimane.html', 'https://www.azlyrics.com/lyrics/guccimane/outro1029399.html', 'https://www.azlyrics.com/lyrics/guccimane/intro.html', 'https://www.azlyrics.com/lyrics/guccimane/traphouse.html', 'https://www.azlyrics.com/lyrics/guccimane/thatsall.html', 'https://www.azlyrics.com/lyrics/guccimane/bootyshorts.html', 'https://www.azlyrics.com/lyrics/guccimane/icy.html', 'https://www.azlyrics.com/lyrics/guccimane/twothangs.html', 'https://www.azlyrics.com/lyrics/guccimane/moneydontmatter.html', 'https://www.azlyrics.com/lyrics/guccimane/thatsmyhood.html', 'https://www.azly

In [45]:
for artist, lp in lyrics_pages.items ():
    assert(len(set(lp)) > 20)

In [46]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For guccimane we have 1089.
The full pull will take for this artist will take 3.02 hours.
For youngthug we have 383.
The full pull will take for this artist will take 1.06 hours.


In [47]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)

In [66]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

#if os.path.isdir("lyrics") : 
    #shutil.rmtree("lyrics/")

    #os.mkdir("lyrics")

In [48]:
path = 'lyrics'


In [ ]:
url_stub = "https://www.azlyrics.com" 
start = time.time()

total_pages = 0 

for artist in lyrics_pages :

    # Use this space to carry out the following steps: 
    
    # 1. Build a subfolder for the artist
    # 2. Iterate over the lyrics pages
    # 3. Request the lyrics page. 
    # 4. Extract the title and lyrics from the page.
    # 5. Write out the title, two returns ('\n'), and the lyrics. Use `generate_filename_from_url to generate the filename. 


    artist_file_path = os.path.join(path, "{}".format(artist))
    if not os.path.exists(artist_file_path):
        os.mkdir(artist_file_path)
    
    for lyrics in lyrics_pages[artist]:
        r = requests.get(lyrics)
             # Don't forget to add a line like `time.sleep(5 + 10*random.random())`
        # to sleep after making the request
        time.sleep(5 + 10*random.random())
        soup_lyrics = BeautifulSoup(r.content)
    
    
        
      
    
    

In [ ]:
for artist in lyrics_pages :
        for lyrics in lyrics_pages[artist]:


        song = soup_lyrics.find('b')
        title = song.find_next('b').get_text()
        lyrics_song = soup_lyrics.find('div', class_ = False, id = False).get_text()
    
        title_new = title + '\n' + '\n'
        open_path = os.path.join(artist_file_path, generate_filename_from_link(lyrics))
        file = open(open_path, 'w')
        file.write(lyrics_song)
        

In [51]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

In [52]:
twitter_files = os.listdir("twitter")
twitter_files = [f for f in twitter_files if f != ".DS_Store"]
artist_handles = list(set([name.split("_")[0] for name in twitter_files]))

print(f"We see two artist handles: {artist_handles[0]} and {artist_handles[1]}.")

We see two artist handles: youngthug and gucci1017.


In [ ]:
for artist in artist_handles :
    follower_file = artist + "_followers.txt"
    follower_data_file = artist + "_followers_data.txt"
    
    ids = open("twitter/" + follower_file,'r').readlines()
    
    print(f"We see {len(ids)-1} in your follower file for {artist}, assuming a header row.")
    
    with open("twitter/" + follower_data_file,'r') as infile :
        
        # check the headers
        headers = infile.readline().split("\t")
        
        print(f"In the follower data file ({follower_data_file}) for {artist}, we have these columns:")
        print(" : ".join(headers))
        
        description_words = []
        locations = set()
        
        
        for idx, line in enumerate(infile.readlines()) :
            line = line.strip("\n").split("\t")
            
            try : 
                locations.add(line[3])            
                description_words.extend(words(line[6]))
            except :
                pass
            print(f"We have {idx+1} data rows for {artist} in the follower data file.")

            print(f"For {artist} we have {len(locations)} unique locations.")

            print(f"For {artist} we have {len(description_words)} words in the descriptions.")
            print("Here are the five most common words:")
            print(Counter(description_words).most_common(5))

        
            print("")
            print("-"*40)
            print("")

In [54]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")

For guccimane we have 119 files.
For guccimane we have roughly 60504 words, 5158 are unique.
For youngthug we have 1 files.
For youngthug we have roughly 0 words, 0 are unique.
